# AgriWeather.ipynb

- 데이터 중에서 농업기상에 관련된 데이터를 수집하고, 전처리하는 파일


In [1]:
import sys
import time
from pprint import pprint


import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format

# DC
import requests
from bs4 import BeautifulSoup

# about file format
import xmltodict


sys.path.append("../Import")
# 'auth' is included in gitignore.
import auth
authkey = auth.authkey["FmlandWthrInfo_dec"]

root_path = "../"
data_path = f"{root_path}data/"

# FmlandWthrInfo

- 작물명  →  (지역코드(AREA_ID), 작물특성코드(PA_CROP_SPE_ID))  ⇒  FmlandWthrInfo

## 기상청_작물별 농업주산지 상세날씨 조회서비스

- 주의사항 : 

- Data Source : [기상청_작물별 농업주산지 상세날씨 조회서비스 - 일통계조회](https://www.data.go.kr/data/15059518/openapi.do)

- Collection Method : API_REST

- Data Format : JSON+XML

- 참고문서 : 작물별_농업주산지_상세날씨_조회서비스__지역코드_22.05.xlsx
    - 지역코드와 작물특성코드 목록

### 데이터 설명

- 농작물(36종) 주산지의 현재 날씨 및 동네예보 정보와 과거날씨 통계(일별, 순별, 월별) 정보 제공

### 요청변수(Request Parameter)

|항목명(국문)|항목명(영문)|항목크기|항목구분|샘플데이터|항목설명|
|---|---|---|---|---|---|
|서비스키|ServiceKey|4|필수|-|공공데이터포털에서 받은 인증키|
|페이지 번호|pageNo|4|필수|1|페이지번호|
|한 페이지 결과 수|numOfRows|4|필수|10|한 페이지 결과 수|
|응답자료형식|dataType|4|옵션|XML|요청자료형식(XML/JSON)|
|시작 연월일|ST_YMD|`0|필수|20161201|일통계 시작 날짜(YYYYMMDD)|
|종료 연월일|ED_YMD|10|필수|20161201|일통계 종료 날짜(YYYYMMDD)|
|지역 아이디|AREA_ID|10|필수|4122000000|지역 아이디(활용가이드 하단첨부 참고)|
|작물별 특성 아이디|PA_CROP_SPE_ID|10|필수|PA130201|주산지 작물별 특성 아이디(활용가이드 하단첨부 참고)|


### 출력결과(Response Element)

|항목명(국문)|항목명(영문)|항목크기|항목구분|샘플데이터|항목설명|
|---|---|---|---|---|---|
|결과코드|resultCode|2|필수|0|결과코드|
|결과메시지|resultMsg|50|필수|OK|결과메시지|
|한 페이지 결과 수|numOfRows|4|필수|10|한 페이지 결과 수|
|페이지 번호|pageNo|4|필수|1|페이지번호|
|전체 결과 수|totalCount|4|필수|3|전체 결과 수|
|데이터 타입|dataType|4|필수|XML|응답자료형식 (XML/JSON)|
|연원일|ymd|10|필수|20161201|일통계 날짜|
|지역 아이디|areaId|10|필수|4827000001|지역 아이디|
|지역 이름|areaName|50|필수|평택|지역 이름|
|작물 명|paCropName|20|필수|무|주산지 작물 이름|
|작물별 특성아이디|paCropSpeId|10|필수|PA020101|작물별 특성 아이디|
|작물별 특성 이름|paCropSpeName|20|필수|봄|주산지 작물별 특성 이름|
|일 평균기온|dayAvgTa|5|필수|24.5|일 평균기온|
|일 최고기온|dayMaxTa|5|필수|30|일 최고기온|
|일 최저기온|dayMinTa|5|필수|22|일 최저기온|
|일 평균상대습도|dayAvgRhm|5|필수|85|일 평균상대습도|
|일 최저상대습도|dayMinRhm|5|필수|60|일 최저상대습도|
|일 강수량|daySumRn|5|필수|25|일 강수량|
|일 평균풍속|dayAvgWs|5|필수|4|일 평균풍속|
|일 누적일조시간|daySumSs|5|필수|4|일 누적일조시간|
|특보 발효 여부|wmCount|4|필수|1|과거특보 발효여부(0:없음, 1:있음)|
|특보 코드|wmCd|8|필수|W2|특보 코드|


# 작물명을 넣으면 지역코드, 작물특성코드 반환하는 함수

- 작물명  →  (지역코드(AREA_ID), 작물특성코드(PA_CROP_SPE_ID))  ⇒  FmlandWthrInfo

## 문제(?)사항

1. 같은 작물에 복수의 작물_특성이 존재함.
    - 작물이 '호박'인 경우에 작물_특성으로 ['애호박', '쥬키니']가 있음.
2. 같은 작물 또는 같은 작물_특성에서 복수의 지역이 존재함.

- 위의 두 이유로 기상데이터를 어떻게 산출(계산?)하는 것이 예측에 좋을지 고민해보아야함. 

### 문제 해결 방안

- 문제1

    1. 모든 작물_특성을 사용.


- 문제2

    1. 모든 지역의 기상 데이터를 평균으로 계산
    2. 모든 지역 중 생산량 상위 n개 또는 n%를 평균 또는 가중치를 부여하여 계산


### 우리팀의 이번 프로젝트에서 선택한 방안과 그 이유

- 선택한 방안

- 선택한 이유

## 지역코드 목록

In [2]:
# 작물별_농업주산지_상세날씨_조회서비스__지역코드_22.05

df_code_raw = pd.read_excel(f"{data_path}작물별_농업주산지_상세날씨_조회서비스__지역코드_22.05.xlsx")
print(df_code_raw.info())
display(df_code_raw.describe())
display(df_code_raw.describe(include="object"))
display(df_code_raw.sample(10))

display(df_code_raw["주산지_작물명"].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 681 entries, 0 to 680
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   지역_아이디         681 non-null    int64 
 1   지역_이름          681 non-null    object
 2   주산지_작물_특성_아이디  681 non-null    object
 3   주산지_작물_특성_이름   681 non-null    object
 4   주산지_작물명        681 non-null    object
dtypes: int64(1), object(4)
memory usage: 26.7+ KB
None


,지역_아이디
count,681.000
mean,4131752619.677
std,775162282.803
min,2611000000.000
25%,4146100000.000
50%,4415000000.000
75%,4688000000.000
max,5013000000.000


,지역_이름,주산지_작물_특성_아이디,주산지_작물_특성_이름,주산지_작물명
count,681,681,681,681
unique,152,62,31,36
top,동구,PA200101,-,배추
freq,16,36,237,63


,지역_아이디,지역_이름,주산지_작물_특성_아이디,주산지_작물_특성_이름,주산지_작물명
178,4148000000,파주,PA230101,일반계,쌀
568,4272000000,홍천,PA080201,고랭지,당근
360,2623000000,부산진구,PA150101,-,방울토마토
457,4217051000,동해,PA120101,한지형,마늘
18,4277000000,정선,PA350301,청양고추,풋고추
437,4276000000,평창,PA130301,고랭지,무
388,4423000000,논산,PA150101,-,방울토마토
285,2671000000,기장군,PA170201,봄,배추
402,2638000000,사하구,PA140101,-,미나리
367,2644000000,강서구,PA150101,-,방울토마토


array(['호박', '풋고추', '포도', '팥', '콩', '참다래', '참깨', '쪽파', '오이', '열무',
       '얼갈이배추', '양파', '양배추', '쌀', '시금치', '생강', '사과', '붉은고추', '복숭아', '배추',
       '배', '방울토마토', '미나리', '무', '마늘', '땅콩', '들깨', '대파', '당근', '단감', '녹두',
       '깻잎', '고구마', '건고추', '감자', '감귤'], dtype=object)

### 탐색

In [3]:
# 컬럼'지역_아이디'가 int형식으로 되어있는데, 자리수가 안맞는 행이 있는지 확인해야함.
# 같으면 글자 수 맞출필요 없음.

df = df_code_raw.copy()

print(
    len(df["지역_아이디"].unique())
    == len(df[df["지역_아이디"].astype(str).str.len() == 10]["지역_아이디"].unique())
)

True


In [4]:
# 한 품목에 대한 id 탐색
print(df[df["주산지_작물명"] == "양배추"]["지역_아이디"].unique())
print()
print(df[df["주산지_작물명"].isin(["양배추"])]["지역_아이디"].unique())
print()
print(df[df["주산지_작물명"] == "양배추"]["주산지_작물_특성_아이디"].unique())

[2711000000 2714000000 2717000000 2720000000 2723000000 2726000000
 2729000000 2771000000 4272000000 4276000000 4277000000 4421000000
 4677000000 4680000000 4682000000 4684000000 4690000000 4719000000
 4775000000 4776000000 4825000000 4827000000 5011000000 5013000000]

[2711000000 2714000000 2717000000 2720000000 2723000000 2726000000
 2729000000 2771000000 4272000000 4276000000 4277000000 4421000000
 4677000000 4680000000 4682000000 4684000000 4690000000 4719000000
 4775000000 4776000000 4825000000 4827000000 5011000000 5013000000]

['PA240101' 'PA240201' 'PA240301' 'PA240401']


### df_code_filtered : crop_list에 있는 품목만 데이터프레임으로 만들기

In [5]:
import definitions

crop_list = definitions.crop_list
print(crop_list)

['호박', '고구마', '양파', '무', '배', '배추', '마늘', '양배추', '시금치', '콩', '땅콩', '풋고추', '오이', '깻잎', '생강', '사과', '포도', '당근', '복숭아', '미나리', '감자', '열무', '쌀']


In [6]:
df_code_filtered = df_code_raw.copy()

df_code_filtered = df[df["주산지_작물명"].isin(crop_list)][["지역_아이디", "주산지_작물_특성_아이디", "주산지_작물명"]]
display(df_code_filtered)

,지역_아이디,주산지_작물_특성_아이디,주산지_작물명
0,4211000000,PA360101,호박
1,4211000000,PA360201,호박
2,4272000000,PA360101,호박
3,4272000000,PA360201,호박
4,4278000000,PA360201,호박
...,...,...,...
673,4827000000,PA020301,감자
674,4887000000,PA020101,감자
675,4888000000,PA020101,감자
676,5011000000,PA020301,감자


In [7]:
# FmlandWthrInfo_code.csv
import os
file_name = "FmlandWthrInfo_code.csv"
df_code_filtered.to_csv(f"{data_path}{file_name}", index=False)
print(file_name in os.listdir(data_path))

True


### fnc : 주산지_작물명 -> (지역_아이디, 주산지_작물_특성_아이디)

In [8]:
def get_code_list(crop_name, file_path=f"../data/FmlandWthrInfo_code.csv"):
    df = pd.read_csv(file_path, index_col=False)
    df = df[df["주산지_작물명"] == crop_name].drop_duplicates(subset=["지역_아이디"])
    v = []
    for idx in df.itertuples():
        v.append(idx[1:3])
    return v
get_code_list("마늘")

[(4215000000, 'PA120101'),
 (4217051000, 'PA120101'),
 (4221051000, 'PA120101'),
 (4223000000, 'PA120101'),
 (4282000000, 'PA120101'),
 (4283032000, 'PA120101'),
 (4380000000, 'PA120101'),
 (4421000000, 'PA120101'),
 (4427000000, 'PA120201'),
 (4481000000, 'PA120301'),
 (4482500000, 'PA120101'),
 (4677000000, 'PA120301'),
 (4682000000, 'PA120301'),
 (4684000000, 'PA120301'),
 (4686000000, 'PA120301'),
 (4689000000, 'PA120301'),
 (4691000000, 'PA120301'),
 (4723000000, 'PA120201'),
 (4772000000, 'PA120101'),
 (4773000000, 'PA120101'),
 (4783000000, 'PA120201'),
 (4824000000, 'PA120301'),
 (4874000000, 'PA120201'),
 (4884000000, 'PA120301'),
 (4889000000, 'PA120201'),
 (5011000000, 'PA120301'),
 (5013000000, 'PA120301')]

# 지역코드 → 농업기상정보

- ? → <u>**지역코드 → 농업기상정보**</u>


In [9]:
# api test
url = "http://apis.data.go.kr/1360000/FmlandWthrInfoService/getDayStatistics"
params = {
    "serviceKey": authkey,
    "pageNo": "1",
    "numOfRows": "10",
    "dataType": "XML",
    "ST_YMD": "20161201",
    "ED_YMD": "20161201",
    "AREA_ID": "4122000000",
    "PA_CROP_SPE_ID": "PA130201",
}

time.sleep(0.2)
response = requests.get(url, params=params)
if response.status_code == 200:
    soup = BeautifulSoup(response.text.encode("utf-8"), "xml")
    soup = soup.find_all("item")[0]

    # soup to dict
    parsed_dict = xmltodict.parse(str(soup))

    # dict to pd.df, check pd.df
    display(pd.DataFrame(parsed_dict).T.reset_index(drop=True))

    # check columns
    print(pd.DataFrame(parsed_dict).T.columns.to_list())

    columns = list(parsed_dict["item"].keys())
    print(columns)

    df_bl = {}
    for i in columns:
        df_bl[i] = []
    print(df_bl)


    # set rename_columns, rename_dict
    rename_columns = [
        "Area_ID",
        "Area_Name",
        "DayAvg_RelativeHumidity",
        "DayAvg_Temperature",
        "dayAvg_WindSpeed",
        "DayMax_Temperature",
        "DayMin_RelativeHumidity",
        "DayMin_Temperature",
        "DaySum_Rainfall",
        "daySum_Sunshine",
        "Crop_Name",
        "CropSpecific_ID",
        "CropSpecific_Name",
        "Warning_Code",
        "Warning_Count",
        "YearMonthDay",
    ]
    rename_dict = {}
    for i, j in zip(columns, rename_columns):
        rename_dict[i] = j
    pprint(rename_dict)

,areaId,areaName,dayAvgRhm,dayAvgTa,dayAvgWs,dayMaxTa,dayMinRhm,dayMinTa,daySumRn,daySumSs,paCropName,paCropSpeId,paCropSpeName,wrnCd,wrnCount,ymd
0,4122000000,평택,59,5,3,9,19,0,0,0,무,PA130201,봄,None,0,2016-12-01 00:00:00


['areaId', 'areaName', 'dayAvgRhm', 'dayAvgTa', 'dayAvgWs', 'dayMaxTa', 'dayMinRhm', 'dayMinTa', 'daySumRn', 'daySumSs', 'paCropName', 'paCropSpeId', 'paCropSpeName', 'wrnCd', 'wrnCount', 'ymd']
['areaId', 'areaName', 'dayAvgRhm', 'dayAvgTa', 'dayAvgWs', 'dayMaxTa', 'dayMinRhm', 'dayMinTa', 'daySumRn', 'daySumSs', 'paCropName', 'paCropSpeId', 'paCropSpeName', 'wrnCd', 'wrnCount', 'ymd']
{'areaId': [], 'areaName': [], 'dayAvgRhm': [], 'dayAvgTa': [], 'dayAvgWs': [], 'dayMaxTa': [], 'dayMinRhm': [], 'dayMinTa': [], 'daySumRn': [], 'daySumSs': [], 'paCropName': [], 'paCropSpeId': [], 'paCropSpeName': [], 'wrnCd': [], 'wrnCount': [], 'ymd': []}
{'areaId': 'Area_ID',
 'areaName': 'Area_Name',
 'dayAvgRhm': 'DayAvg_RelativeHumidity',
 'dayAvgTa': 'DayAvg_Temperature',
 'dayAvgWs': 'dayAvg_WindSpeed',
 'dayMaxTa': 'DayMax_Temperature',
 'dayMinRhm': 'DayMin_RelativeHumidity',
 'dayMinTa': 'DayMin_Temperature',
 'daySumRn': 'DaySum_Rainfall',
 'daySumSs': 'daySum_Sunshine',
 'paCropName': 'Cro

In [10]:
def Get_df_AgriWeather(
    authkey: str,
    pageNo: int | str,
    search_Year_Period: tuple,
    AREA_ID: str,
    PA_CROP_SPE_ID: str,
) -> pd.DataFrame | int:

    """
    API에서 농업기상정보를 가져오는 함수
    단순하게 API에서 요구하는 'Page_Size'를 제외한 요청변수를 인자로 받음.
    """

    url = "http://apis.data.go.kr/1360000/FmlandWthrInfoService/getDayStatistics"
    params = {
        "serviceKey": authkey,
        "pageNo": pageNo,
        "numOfRows": 100,
        "dataType": "XML",
        "ST_YMD": f"{search_Year_Period[0]}0101",
        "ED_YMD": f"{search_Year_Period[1]}1231",
        "AREA_ID": AREA_ID,
        "PA_CROP_SPE_ID": PA_CROP_SPE_ID,
    }

    # API
    time.sleep(0.01)
    response = requests.get(url, params)

    df = pd.DataFrame(
        {
            "areaId": [],
            "areaName": [],
            "dayAvgRhm": [],
            "dayAvgTa": [],
            "dayAvgWs": [],
            "dayMaxTa": [],
            "dayMinRhm": [],
            "dayMinTa": [],
            "daySumRn": [],
            "daySumSs": [],
            "paCropName": [],
            "paCropSpeId": [],
            "paCropSpeName": [],
            "wrnCd": [],
            "wrnCount": [],
            "ymd": [],
        }
    )

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "xml")
        soup = soup.find_all("item")
        try:
            s = soup[-1]

            # col
            col = xmltodict.parse(str(s))
            col = list(col["item"].keys())

            # # no
            # no = int(s.find("no").get_text())
            # if no >= 301:
            #     roof_size = no % 100
            # else:
            #     roof_size = 100
        except:
            return None

        for i in range(0, 100):
            try:
                s = soup[i]
                temp_dict = {"temp_idx": {}}
            except:
                continue

            for j in col:
                try:
                    v = s.find(j).get_text()
                    temp_dict["temp_idx"][j] = v
                except:
                    pass

            df_new = pd.DataFrame(temp_dict).T
            df = pd.concat([df, df_new]).reset_index(drop=True)

        return df[col]

    else:
        print(response.status_code)
        return response.status_code

In [29]:
def Get_df_PeriodAgriWeather(
    authkey: str,
    search_Year_Period: tuple,
    AREA_ID: str,
    PA_CROP_SPE_ID: str,
) -> pd.DataFrame:

    """
    'Get_df_AgriWeather' 함수를 활용하여, 더 편하게(?) 농업기상정보를 가져오는 함수
    """

    df = pd.DataFrame()

    for y in range(search_Year_Period[0], search_Year_Period[1] + 1):
        for p in range(1, 5):
            df_new = Get_df_AgriWeather(
                authkey=authkey,
                pageNo=p,
                search_Year_Period=search_Year_Period,
                AREA_ID=AREA_ID,
                PA_CROP_SPE_ID=PA_CROP_SPE_ID,
            )
            df = pd.concat([df, df_new])

    # 대략적인 전처리

    # 시간 지우기
    # df["ymd"] = df["ymd"].str.replace(" 00:00:00", "")

    # rename_col
    rename_dict = {
        "ymd": "YMD",
        "areaId": "Area_ID",
        "areaName": "Area_Name",
        "paCropName": "Crop_Name",
        "paCropSpeId": "CropSpecific_ID",
        "paCropSpeName": "CropSpecific_Name",
        "dayAvgTa": "DayAvg_Temperature",
        "dayMaxTa": "DayMax_Temperature",
        "dayMinTa": "DayMin_Temperature",
        "dayAvgRhm": "DayAvg_RelativeHumidity",
        "dayMinRhm": "DayMin_RelativeHumidity",
        "daySumRn": "DaySum_Rainfall",
        "dayAvgWs": "DayAvg_WindSpeed",
        "daySumSs": "DaySum_Sunshine",
        "wrnCount": "Warning_Count",
        "wrnCd": "Warning_Code",
    }

    try:
        df = df[list(rename_dict.keys())].rename(columns=rename_dict)
    except:
        pass

    # reorder_col
    col_list = [
        "YMD",
        "Crop_Name",
        "DayAvg_Temperature",
        "DayMax_Temperature",
        "DayMin_Temperature",
        "DayAvg_RelativeHumidity",
        "DayMin_RelativeHumidity",
        "DaySum_Rainfall",
        "DayAvg_WindSpeed",
        "DaySum_Sunshine",
        "Warning_Count",
        "Warning_Code",
    ]

    try:
        df = df[col_list]
    except:
        pass


    return df


# display(
#     Get_df_PeriodAgriWeather(
#         authkey=authkey,
#         search_Year_Period=(2018, 2019),
#         AREA_ID="4215000000",
#         PA_CROP_SPE_ID="PA120101",
#     )
# )

In [30]:
def Get_AgriWeather_All(
    authkey,
    search_Year_Period,
    crop_name,
    code_file_path=f"../data/FmlandWthrInfo_code.csv",
    avg=True,
):

    code_list = get_code_list(crop_name=crop_name, file_path=code_file_path)

    df = pd.DataFrame()

    for idtuple in code_list:
        df_new = Get_df_PeriodAgriWeather(
            authkey=authkey,
            search_Year_Period=search_Year_Period,
            AREA_ID=idtuple[0],
            PA_CROP_SPE_ID=idtuple[1],
        )

        df = pd.concat([df, df_new])

    if avg == True:

        dict = {
            "DayAvg_Temperature": "float",
            "DayMax_Temperature": "float",
            "DayMin_Temperature": "float",
            "DayAvg_RelativeHumidity": "float",
            "DayMin_RelativeHumidity": "float",
            "DaySum_Rainfall": "float",
            "DayAvg_WindSpeed": "float",
            "DaySum_Sunshine": "float",
            "Warning_Count": "int",
        }
        
        df = df.astype(dict)

        df["DayDiff_Temperature"] = (
            df["DayMax_Temperature"] - df["DayMin_RelativeHumidity"]
        )

        col_list = [
            "YMD",
            "Crop_Name",
            "DayAvg_Temperature",
            "DayDiff_Temperature",
            # "DayMax_Temperature",
            # "DayMin_Temperature",
            "DayAvg_RelativeHumidity",
            # "DayMin_RelativeHumidity",
            "DaySum_Rainfall",
            "DayAvg_WindSpeed",
            "DaySum_Sunshine",
            "Warning_Count",
            # "Warning_Code",
        ]

        df = df.groupby("YMD")[col_list].mean()

    return df


# df_api = Get_AgriWeather_All(authkey, search_Year_Period=(2015, 2015), crop_name="마늘")
# df_api

# 만든 함수로 데이터 가져오기

In [42]:
from definitions import crop_list2

print(crop_list2)

# crop_list2.remove('시금치')

print(crop_list2)

['마늘', '깻잎', '감자', '고구마']
['마늘', '깻잎', '감자', '고구마']


In [ ]:
crop_name = "고구마"

for i in range(2005, 2021):
    df_api = Get_AgriWeather_All(authkey, search_Year_Period=(i, i), crop_name=crop_name)
    df_api.to_csv(f"{data_path}AgriWeather/AgriWeather_{crop_name}_{i}.csv")

In [ ]:
crop_name = "깻잎"

for i in range(2005, 2021):
    df_api = Get_AgriWeather_All(authkey, search_Year_Period=(i, i), crop_name=crop_name)
    df_api.to_csv(f"{data_path}AgriWeather/AgriWeather_{crop_name}_{i}.csv")

['../data/AgriWeather\\AgriWeather_마늘_2005.csv',
 '../data/AgriWeather\\AgriWeather_마늘_2006.csv',
 '../data/AgriWeather\\AgriWeather_마늘_2007.csv',
 '../data/AgriWeather\\AgriWeather_마늘_2008.csv',
 '../data/AgriWeather\\AgriWeather_마늘_2009.csv',
 '../data/AgriWeather\\AgriWeather_마늘_2010.csv',
 '../data/AgriWeather\\AgriWeather_마늘_2011.csv',
 '../data/AgriWeather\\AgriWeather_마늘_2012.csv',
 '../data/AgriWeather\\AgriWeather_마늘_2013.csv',
 '../data/AgriWeather\\AgriWeather_마늘_2014.csv',
 '../data/AgriWeather\\AgriWeather_마늘_2015.csv',
 '../data/AgriWeather\\AgriWeather_마늘_2016.csv',
 '../data/AgriWeather\\AgriWeather_마늘_2017.csv',
 '../data/AgriWeather\\AgriWeather_마늘_2018.csv',
 '../data/AgriWeather\\AgriWeather_마늘_2019.csv',
 '../data/AgriWeather\\AgriWeather_마늘_2020.csv']

In [56]:
from glob import glob
# ['마늘', '깻잎', '감자', '고구마']

crop_name = "고구마"

files_path = f"{data_path}AgriWeather/"

files = glob(f"{files_path}/AgriWeather_{crop_name}*")
print(files)

df = pd.DataFrame()
for file in files:
    df_n = pd.read_csv(f'{file}')
    df = pd.concat([df, df_n])

df.to_csv(f"{data_path}AgriWeather/AgriWeather_{crop_name}.csv")

['../data/AgriWeather\\AgriWeather_고구마_2005.csv', '../data/AgriWeather\\AgriWeather_고구마_2006.csv', '../data/AgriWeather\\AgriWeather_고구마_2007.csv', '../data/AgriWeather\\AgriWeather_고구마_2008.csv', '../data/AgriWeather\\AgriWeather_고구마_2009.csv', '../data/AgriWeather\\AgriWeather_고구마_2010.csv', '../data/AgriWeather\\AgriWeather_고구마_2011.csv', '../data/AgriWeather\\AgriWeather_고구마_2012.csv', '../data/AgriWeather\\AgriWeather_고구마_2013.csv', '../data/AgriWeather\\AgriWeather_고구마_2014.csv', '../data/AgriWeather\\AgriWeather_고구마_2015.csv', '../data/AgriWeather\\AgriWeather_고구마_2016.csv', '../data/AgriWeather\\AgriWeather_고구마_2017.csv', '../data/AgriWeather\\AgriWeather_고구마_2018.csv', '../data/AgriWeather\\AgriWeather_고구마_2019.csv', '../data/AgriWeather\\AgriWeather_고구마_2020.csv']
